<a href="https://colab.research.google.com/github/erdemustun/dosya/blob/main/999_Banker_Dip_Taramas%C4%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/rongardF/tvdatafeed tradingview-screener

import numpy as np
import pandas as pd
from tvDatafeed import TvDatafeed, Interval
from tradingview_screener import get_all_symbols
import warnings

warnings.simplefilter(action='ignore')
def xsa(src, length, weight):
    sumf = np.zeros_like(src)
    ma = np.zeros_like(src)
    out = np.zeros_like(src)

    for i in range(length,len(src)):
        sumf[i] = np.nan_to_num(sumf[i - 1]) - np.nan_to_num(src[i - length]) + src[i]
        ma[i] = np.nan if np.isnan(src[i - length]) else sumf[i] / length
        out[i] = ma[i] if np.isnan(out[i - 1]) else (src[i] * weight + out[i - 1] * (length - weight)) / length

    return out

def ema(src, span):
    return src.ewm(span=span, adjust=False).mean()

def Banker_Fund_Trend(data):
    df = data.copy()
    close = df['close']
    low = df['low']
    high = df['high']
    open_ = df['open']
    fundtrend =np.zeros_like(close)
    part_1 = (close - low.rolling(window=27).min()) / (high.rolling(window=27).max() - low.rolling(window=27).min()) * 100
    fundtrend = ((3 * xsa(part_1, 5, 1) - 2 * xsa(xsa(part_1, 5, 1), 3, 1) - 50) * 1.032 + 50)

    typ = (2 * close + high + low + open_) / 5
    lol = low.rolling(window=34).min()
    hoh = high.rolling(window=34).max()

    bullbearline = ema((typ - lol) / (hoh - lol) * 100, 13)
    bankerentry = (fundtrend > bullbearline) & (bullbearline < 25)

    df['fundtrend'] = fundtrend
    df['bullbearline'] = bullbearline
    df['bankerentry'] = bankerentry.astype(int)  # Convert to integer

    return df


tv = TvDatafeed()
Hisseler = get_all_symbols(market='turkey')
Hisseler = [symbol.replace('BIST:', '') for symbol in Hisseler]
Hisseler = sorted(Hisseler)


#Raporlama için kullanılacak başlıklar
Titles = ['Hisse Adı', 'Son Fiyat','Dip Sinyali']
df_signals = pd.DataFrame(columns=Titles)

for hisse in Hisseler:
    try:
        data = tv.get_hist(symbol=hisse, exchange='BIST', interval=Interval.in_daily, n_bars=100)
        data = data.reset_index()
        Banker = Banker_Fund_Trend(data)
        Banker.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'}, inplace=True)
        Banker.set_index('datetime', inplace=True)
        Buy = False
        Signals = Banker.tail(2)
        Signals = Signals.reset_index()

        Entry = (Signals.loc[0, 'bankerentry'] == False) & (Signals.loc[1, 'bankerentry'] == True)
        Last_Price = Signals.loc[1, 'Close']
        L1 = [hisse, Last_Price,Entry]
        df_signals.loc[len(df_signals)] = L1
        print(L1)
    except:
        pass

df_True = df_signals[(df_signals['Dip Sinyali'] == True)]
print(df_True)

  Cloning https://github.com/rongardF/tvdatafeed to /tmp/pip-req-build-ug36gykc
  Running command git clone --filter=blob:none --quiet https://github.com/rongardF/tvdatafeed /tmp/pip-req-build-ug36gykc
  Resolved https://github.com/rongardF/tvdatafeed to commit e6f6aaa7de439ac6e454d9b26d2760ded8dc4923
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 692.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 22.3 MB/s eta 0:00:00
  Created wheel for tvdatafeed: filename=tvdatafeed-2.1.0-py3-none-any.whl size=17533 sha256=11cd5c6af2dfa512d2b5f3c79f63d8f9a542a4a50f4c45a2568b817709fbe5f5
  Stored in directory: /tmp/pip-ephem-wheel-cache-idppmr79/wheels/e4/32/1e/21ebcacc6549d75fae3bf3ff75cee8fcbe4e5c189d88fbd5a1
Successfully built tvdatafeed
  Attempting uninstall: pandas
    Found existing installation: pandas 2.0.3
    Uninstalling pandas-2.0.3:
      Successfully uninstalled pandas-2.0.3
ERROR: pip's dependency res

['A1CAP', 26.799999, False]
['ACSEL', 135.0, False]
['ADEL', 571.5, False]
['ADESE', 2.29, False]
['ADGYO', 27.66, False]
['AEFES', 211.0, False]
['AFYON', 12.97, False]
['AGESA', 113.3, False]
['AGHOL', 352.75, False]
['AGROT', 16.66, False]
['AGYO', 8.15, False]
['AHGAZ', 13.76, False]
['AKBNK', 61.150002, False]
['AKCNS', 132.89999, False]
['AKENR', 17.25, True]
['AKFGY', 2.36, False]
['AKFYE', 27.0, False]
['AKGRT', 8.55, False]
['AKMGY', 290.25, False]
['AKSA', 124.4, False]
['AKSEN', 45.860001, False]
['AKSGY', 17.27, False]
['AKSUE', 13.35, False]
['AKYHO', 7.61, False]
['ALARK', 118.1, False]
['ALBRK', 5.5, False]
['ALBTS', 2662.0, False]
['ALCAR', 1340.0, False]
['ALCTL', 107.8, False]
['ALDUK', 6950.0, False]
['ALFAS', 77.050003, True]
['ALGYO', 42.560001, False]
['ALKA', 28.059999, False]
['ALKIM', 36.299999, False]
['ALKLC', 30.540001, False]
['ALMAD', 6.47, False]
['ALTIN', 23.99, False]
['ALTNY', 92.050003, False]
['ALVES', 42.599998, False]
['ANELE', 18.620001, False]
['